In [1]:

# Import necessary libraries
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score,log_loss,r2_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV,train_test_split ,StratifiedKFold,KFold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import VotingClassifier, BaggingClassifier,BaggingRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.neighbors import *
from sklearn.svm import SVC
from sklearn.pipeline import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os

warnings.simplefilter('ignore')

os.chdir('D:\kaggla_comp\hardness')

train = pd.read_csv('train.csv')
train

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,0,100.0,0.841611,10.000000,4.800000,20.612526,11.088100,2.766000,1.732000,0.860000,0.496070,0.91457,6.0
1,1,100.0,7.558488,10.000000,4.800000,20.298893,12.040830,2.755000,1.631000,0.910000,0.492719,0.71760,6.5
2,2,76.0,8.885992,15.600000,5.600000,33.739258,12.086300,2.828000,1.788000,0.864000,0.481478,1.50633,2.5
3,3,100.0,8.795296,10.000000,4.800000,20.213349,10.948500,2.648000,1.626000,0.936000,0.489272,0.78937,6.0
4,4,116.0,9.577996,11.600000,4.800000,24.988133,11.824480,2.766000,1.682000,0.896000,0.492736,1.86481,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10402,10402,128.0,7.558488,12.000000,4.000000,26.385218,11.330440,2.644000,1.631000,0.892000,0.496070,1.79607,4.0
10403,10403,30.0,1.743160,10.000000,5.333333,20.766935,14.163933,3.090000,1.556667,0.866667,0.480390,0.81480,5.0
10404,10404,196.0,30.920000,24.500000,5.500000,53.490297,10.074300,2.295000,1.545000,1.120000,0.469715,2.11540,1.8
10405,10405,38.0,1.553160,12.666667,4.666667,26.621687,11.290033,2.743333,1.756667,0.980000,0.486507,0.77755,6.0


In [3]:
x= train.drop('Hardness',axis=1)
y=train['Hardness']

In [4]:
# Define the parameter grid for hyperparameter tuning
params = {
    'max_features': [3, 4, 5, 6, 7],
    'max_depth': [None, 3, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

# Create a K-Fold cross-validation object with 5 splits, shuffling the data, and a random state of 24
kfold = KFold(n_splits=5, shuffle=True, random_state=24)

# Create a Random Forest Regressor with a random state of 24
rf = RandomForestRegressor(random_state=24)

# Create a GridSearchCV object to perform cross-validation and hyperparameter tuning
# Use the Random Forest Regressor as the estimator, the parameter grid defined above, the K-Fold cross-validation,
# set the verbosity to 3
gcv = GridSearchCV(rf, param_grid=params, verbose=3, cv=kfold)

# Fit the GridSearchCV object to the entire dataset (x and y)
gcv.fit(x, y)

# Print the best parameters found by the GridSearchCV
print("Best Parameters:", gcv.best_params_)

# Print the best score found by the GridSearchCV
print("Best Score:", gcv.best_score_)

Fitting 5 folds for each of 135 candidates, totalling 675 fits
[CV 1/5] END max_depth=None, max_features=3, min_samples_leaf=1, min_samples_split=2;, score=0.479 total time=   2.6s
[CV 2/5] END max_depth=None, max_features=3, min_samples_leaf=1, min_samples_split=2;, score=0.453 total time=   2.5s
[CV 3/5] END max_depth=None, max_features=3, min_samples_leaf=1, min_samples_split=2;, score=0.456 total time=   2.5s
[CV 4/5] END max_depth=None, max_features=3, min_samples_leaf=1, min_samples_split=2;, score=0.464 total time=   2.5s
[CV 5/5] END max_depth=None, max_features=3, min_samples_leaf=1, min_samples_split=2;, score=0.480 total time=   2.5s
[CV 1/5] END max_depth=None, max_features=3, min_samples_leaf=1, min_samples_split=5;, score=0.481 total time=   2.2s
[CV 2/5] END max_depth=None, max_features=3, min_samples_leaf=1, min_samples_split=5;, score=0.459 total time=   2.3s
[CV 3/5] END max_depth=None, max_features=3, min_samples_leaf=1, min_samples_split=5;, score=0.465 total time= 

In [7]:
best_model = gcv.best_estimator_


test_data = pd.read_csv("test.csv")

pred_berry = best_model.predict(test_data)


submit = pd.read_csv("sample_submission.csv")


submit['Hardness'] = pred_berry

# Save the submission file with the predicted hardness values
submit.to_csv("D:\\kaggla_comp\\hardness\\random_forest_hard_25.csv", index=False)